In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
rawdata= pd.read_csv('data/raw/original_data.csv')

In [9]:
rawdata.head()

,Unnamed: 0,City,Business Name,Address,Phone Number,Website,Google Map ID,Review Text,Timestamp,Stars,TrueTimestamp
0,0,adrej,BMCE BANK El Wafaa,"225C+WRW, Fès 30050",NaN,https://business.google.com/create?fp=34034433...,NaN,GAb est sale\r\nPrès d'entrée d'immeuble et bo...,il y a 2 ans,1,2023-03-14 21:22:17.359491
1,1,adrej,BMCE BANK El Wafaa,"225C+WRW, Fès 30050",NaN,https://business.google.com/create?fp=34034433...,NaN,Chef d'agence (une femme) est une hypocrite.,il y a 9 ans,2,2016-03-15 21:22:17.359491
2,2,adrej,BMCE BANK El Wafaa,"225C+WRW, Fès 30050",NaN,https://business.google.com/create?fp=34034433...,NaN,Bonne services,il y a 4 ans,4,2021-03-14 21:22:17.359491
3,3,adrej,BMCE BANK El Wafaa,"225C+WRW, Fès 30050",NaN,https://business.google.com/create?fp=34034433...,NaN,le pire service que j'ai jamais vu dans une ba...,il y a 2 ans,1,2023-03-14 21:22:17.359491
4,4,adrej,BMCE BANK El Wafaa,"225C+WRW, Fès 30050",NaN,https://business.google.com/create?fp=34034433...,NaN,No review text found,il y a 8 mois,5,2024-07-16 21:22:17.359491


In [10]:
#remove column
rawdata= rawdata.drop(columns=['Address','Timestamp',"Phone Number","Unnamed: 0",'Website','Google Map ID'])

remove rows where there is nor review text

In [11]:
rawdata = rawdata[rawdata["Review Text"] != "No review text found"]
rawdata.head()

,City,Business Name,Review Text,Stars,TrueTimestamp
0,adrej,BMCE BANK El Wafaa,GAb est sale\r\nPrès d'entrée d'immeuble et bo...,1,2023-03-14 21:22:17.359491
1,adrej,BMCE BANK El Wafaa,Chef d'agence (une femme) est une hypocrite.,2,2016-03-15 21:22:17.359491
2,adrej,BMCE BANK El Wafaa,Bonne services,4,2021-03-14 21:22:17.359491
3,adrej,BMCE BANK El Wafaa,le pire service que j'ai jamais vu dans une ba...,1,2023-03-14 21:22:17.359491
13,adrej,بريد بنك,"Le gérant prend les rênes de l'agence, et au f...",1,2024-08-15 21:22:17.360486


In [12]:
list(rawdata['Business Name'].unique())

['BMCE BANK El Wafaa',
 'بريد بنك',
 'Al Akhdar Bank',
 'Agence Bank of Africa',
 'Attijariwafa Bank',
 'Société Générale Bank',
 'Bank of Africa',
 'Banque Populaire',
 'CIH Bank - Agence Mohammed V',
 'Poste Maroc - Al Barid Bank',
 'Crédit Agricole',
 'BMCI',
 'CIH Bank- Agence Dakhla',
 'Bank Al-Maghrib',
 'Société Générale',
 'Bank Al Yousr',
 'Dar Al Amane Bank',
 'Attijariwafa Bank - El Houda',
 'Umnia Bank Agadir Kettani',
 'CIH Bank',
 'Poste Maroc - Al Barid Bank - Barid Cash',
 'Attijariwafa Bank - Agence Essalam Expansion',
 'CIH Bank - Agence El Houda',
 'Bank of Africa Talborjt',
 'Bank of Africa - Agence Hassan 1er',
 'Attijariwafa Bank - Agence Agadir Cadi Ayad',
 'CIH Bank - Agence Hay Salam',
 'Arab Bank',
 "Centre d'estivage Banque Populaire",
 'Bank of Africa - Batoir',
 'Banque Populaire Siège Centre Sud',
 'Bank Assafa',
 'CIH Bank - Agence Bensergao',
 'Attijariwafa Bank - Agence Agadir Hay Dakhla',
 'Banque Populaire - Agence Bouabid',
 'Bank of Africa - Agence 

In [13]:
from rapidfuzz import process, fuzz

# Define known bank names
bank_names = {
    "CIH": ["CIH",'Cih','Banque CIH','CIH','CIH bank','CIH Bank - Agence Mohammed V','CIH Bank- Agence Dakhla',
            'CIH Bank - Agence Hay Salam','CIH Bank - Agence Bensergao','CIH BANK','CIH Banque','CIH Bank - Agence'],
    "Attijariwafa Bank": ["Attijariwafa Bank","Attijari", "Attijari Wafabank", "attijari fes", "attijari bank",
                          'Attijariwafa Bank - El Houda','Attijariwafa Bank - Agence Essalam Expansion',
                          'Attijariwafa Bank - Agence Agadir Cadi Ayad','Attijariwafa Bank - Agence Agadir Hay Dakhla',
                          'Attijariwafa Bank 11 Janvier','Attijari Wafa Banque','Attijariwafa bank','Attijari wafa bank'],
    "BMCE Bank": ["BMCE BANK El Wafaa", "BMCE Bank", "bmce fes", "bmcebank"],
    "Banque Populaire": ["Banque Populaire","BANQUE POPULAIRE - Centre d'Affaire Fes Taza",'banque populaire',"Banque Populaire",'Banque Chaabi',
                          "B. Populaire", "populaire fes","Centre d'estivage Banque Populaire",'Banque Populaire Siège Centre Sud',
                          'Banque Populaire - Agence Bouabid','Banque Populaire - Agence Riad Salam','Banque Populaire البنك الشعبي','Banque Populaire.','Groupe Scolaire de La Fondation Banque Populaire_Tanger',
                          'La banque populaire','banque populaire','Banque populaire','Agence Banque Populaire',
                          'Banque Poulaire','Banque populaire Admime','Banque Populaire -Agence Carrefour'],
    "Al Barid Bank": ["Al Barid Bank",'Poste Maroc - Al Barid Bank','Poste Maroc - Al Barid Bank - Barid Cash','Barid Bank',
                      'BARID BANK','Al Barid Bank Fès Narjis','Al Barid Bank'],
    "Bank of Africa":["Bank of Africa",'Agence Bank of Africa','Bank of Africa - Agence Hassan 1er','Bank of Africa Talborjt',
                      'Bank of Africa - Batoir','Bank of Africa - Agence Bensergao','Bank of Africa (BMCE)',
                      'BMCE','bmce bank','Banque BMCE', 'Bank of Africa Agdal'],
    'Bank Assafa': ['Bank Assafa'],
    "BMCI":['BMCI Bank'"BMCI"],
    "Arab Bank": ["Arab Bank"],
    "Bank Al-Maghrib": ["Bank Al-Maghrib"],
    "Bank Al Yousr": ["Bank Al Yousr"],
    'Assafa bank':['Assafa bank'],
    "Crédit Agricole": ['Crédit Agricole Du Maroc','crédit agricole','CREDIT AGRICOLE'],
    "Société Générale": ['Société Générale Bank','Espace Libre Service - Société Générale Maroc','Société Générale GAB','Société Générale Temara Massira']
}

# Function to match business names
def match_bank(name):
    name = name.lower().strip()  # Normalize input
    best_match = "Other"
    highest_score = 0

    for bank, variations in bank_names.items():
        if not variations:  # Skip empty lists
            continue

        match, score, _ = process.extractOne(name, variations, scorer=fuzz.partial_ratio)

        if match and score > highest_score:  # Ensure match is valid
            best_match = bank
            highest_score = score

    return best_match if highest_score >= 80 else "Other"

# Apply function to the dataset
rawdata["Business Name"] = rawdata["Business Name"].apply(match_bank)




In [14]:
#Making sure every rating is within the range 1-5
rawdata["Stars"].describe()

count    8899.000000
mean        2.339139
std         1.710455
min         1.000000
25%         1.000000
50%         1.000000
75%         4.000000
max         5.000000
Name: Stars, dtype: float64

Let's work with only few big cities

In [15]:
# List of valid Moroccan cities
valid_cities = [
    "agadir","ifrane","dakhla","fes","errachidia","casablanca","berkane","azilal","harhoura","el+jadida",
]

rawdata["City"] = rawdata["City"].str.lower().str.strip()

# Remove rows where the city is not in the valid list
rawdata = rawdata[rawdata["City"].isin(valid_cities)]
list(rawdata["City"].unique())


['agadir',
 'azilal',
 'berkane',
 'casablanca',
 'dakhla',
 'el+jadida',
 'errachidia',
 'fes',
 'harhoura',
 'ifrane']

In [16]:
rawdata.describe()

,Stars
count,3475.000000
mean,2.265612
std,1.686398
min,1.000000
25%,1.000000
50%,1.000000
75%,4.000000
max,5.000000


In [17]:
rawdata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3475 entries, 46 to 13252
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   City           3475 non-null   object
 1   Business Name  3475 non-null   object
 2   Review Text    3475 non-null   object
 3   Stars          3475 non-null   int64 
 4   TrueTimestamp  3475 non-null   object
dtypes: int64(1), object(4)
memory usage: 162.9+ KB


In [18]:
rawdata.head()

,City,Business Name,Review Text,Stars,TrueTimestamp
46,agadir,Attijariwafa Bank,Beaucoup des clients qui attends outside alors...,2,2022-03-14 21:22:17.360486
47,agadir,Attijariwafa Bank,Bon service,4,2023-03-14 21:22:17.360486
48,agadir,Attijariwafa Bank,Service👍🏽👍🏽👍🏽👍🏽👍🏽👍🏽 …,5,2024-12-13 21:22:17.360486
50,agadir,Attijariwafa Bank,"Le service est nul, j'ai pas de compte bancair...",1,2024-10-14 21:22:17.360486
51,agadir,Attijariwafa Bank,Excellent. Très bonne banque !\r\nRemarque les...,5,2024-09-14 21:22:17.360486


SAVING TO A NEW CSV


In [32]:
rawdata['sentiment'] = rawdata['Stars'].apply(lambda x: 1 if x >= 3 else 0)
rawdata.to_csv("../data/cleaned/updated_dataset.csv", index=False)
rawdata.head()

,City,Business Name,Review Text,Stars,TrueTimestamp,sentiment
46,agadir,Attijariwafa Bank,Beaucoup des clients qui attends outside alors...,2,2022-03-14 21:22:17.360486,0
47,agadir,Attijariwafa Bank,Bon service,4,2023-03-14 21:22:17.360486,1
48,agadir,Attijariwafa Bank,Service👍🏽👍🏽👍🏽👍🏽👍🏽👍🏽 …,5,2024-12-13 21:22:17.360486,1
50,agadir,Attijariwafa Bank,"Le service est nul, j'ai pas de compte bancair...",1,2024-10-14 21:22:17.360486,0
51,agadir,Attijariwafa Bank,Excellent. Très bonne banque !\r\nRemarque les...,5,2024-09-14 21:22:17.360486,1


In [34]:
df_star = rawdata[["City", "Business Name", "Stars", "TrueTimestamp","sentiment"]]  # Star Dataset
df_review = rawdata[["City", "Business Name", "Review Text", "TrueTimestamp","sentiment"]]  # Review Dataset

# Save both datasets
df_star.to_csv("../data/cleaned/star_dataset.csv", index=False)
df_review.to_csv("../data/cleaned/review_dataset.csv", index=False)

# Show first few rows for confirmation
print("Star Dataset:")
print(df_star.head())

print("\nReview Dataset:")
df_review.head()

Star Dataset:
      City      Business Name  Stars               TrueTimestamp  sentiment
46  agadir  Attijariwafa Bank      2  2022-03-14 21:22:17.360486          0
47  agadir  Attijariwafa Bank      4  2023-03-14 21:22:17.360486          1
48  agadir  Attijariwafa Bank      5  2024-12-13 21:22:17.360486          1
50  agadir  Attijariwafa Bank      1  2024-10-14 21:22:17.360486          0
51  agadir  Attijariwafa Bank      5  2024-09-14 21:22:17.360486          1

Review Dataset:


,City,Business Name,Review Text,TrueTimestamp,sentiment
46,agadir,Attijariwafa Bank,Beaucoup des clients qui attends outside alors...,2022-03-14 21:22:17.360486,0
47,agadir,Attijariwafa Bank,Bon service,2023-03-14 21:22:17.360486,1
48,agadir,Attijariwafa Bank,Service👍🏽👍🏽👍🏽👍🏽👍🏽👍🏽 …,2024-12-13 21:22:17.360486,1
50,agadir,Attijariwafa Bank,"Le service est nul, j'ai pas de compte bancair...",2024-10-14 21:22:17.360486,0
51,agadir,Attijariwafa Bank,Excellent. Très bonne banque !\r\nRemarque les...,2024-09-14 21:22:17.360486,1


SPLITTING THE DATASET INTO 2 : ONE FOR STARS OTHER ONE FOR REVIEWS

1️⃣ Star Dataset → For numerical analysis of ratings.
2️⃣ Review Dataset → For NLP & sentiment analysis